In [188]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('transaction.csv')
dataset["InvoiceDate"] = pd.to_datetime(dataset["InvoiceDate"])
dataset["Month"] = dataset["InvoiceDate"].dt.month
dataset["Year"] = dataset["InvoiceDate"].dt.year
dataset

,InvoiceNo,StockCode,Qty,InvoiceDate,CustomerID,Country,Month,Year
0,537626,22725,830,2010-12-07 14:57:00,12347,Iceland,12,2010
1,537626,22729,948,2010-12-07 14:57:00,12347,Iceland,12,2010
2,537626,22195,695,2010-12-07 14:57:00,12347,Iceland,12,2010
3,542237,22725,636,2011-01-26 14:30:00,12347,Iceland,1,2011
4,542237,22729,536,2011-01-26 14:30:00,12347,Iceland,1,2011
...,...,...,...,...,...,...,...,...
10541,543911,21700,455,2011-02-14 12:46:00,17829,United Arab Emirates,2,2011
10542,543911,22111,578,2011-02-14 12:46:00,17829,United Arab Emirates,2,2011
10543,543911,22112,163,2011-02-14 12:46:00,17829,United Arab Emirates,2,2011
10544,564428,23296,545,2011-08-25 11:27:00,17844,Canada,8,2011


In [189]:
data = dataset[dataset["Year"] == 2011][["InvoiceNo","Country","Month","Year"]]
data

,InvoiceNo,Country,Month,Year
3,542237,Iceland,1,2011
4,542237,Iceland,1,2011
5,542237,Iceland,1,2011
6,542237,Iceland,1,2011
7,542237,Iceland,1,2011
...,...,...,...,...
10541,543911,United Arab Emirates,2,2011
10542,543911,United Arab Emirates,2,2011
10543,543911,United Arab Emirates,2,2011
10544,564428,Canada,8,2011


In [190]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid

transaksi_tinggi = []
nCluster = 3

for idx in range(1, 13):
    data_bulan = data[data["Month"] == idx]
    data_bulan = data_bulan.drop_duplicates(subset="InvoiceNo")
    transaksi = data_bulan.groupby(["Country"])["InvoiceNo"].agg('count').reset_index()

    X = transaksi[["InvoiceNo"]]
    cluster = AgglomerativeClustering(n_clusters=nCluster, affinity="euclidean", linkage="average")
    ypred = cluster.fit_predict(X)
    nearestCentroid = NearestCentroid()
    nearestCentroid.fit(X, ypred)

    centroids = pd.DataFrame(columns=["X"], data=nearestCentroid.centroids_).sort_values(by=["X"])
    transaksi_tinggi.append([idx,centroids.iloc[nCluster-1]["X"]])

transaksi_tinggi = pd.DataFrame(columns=["Bulan","Perkembangan_Transaksi"], data=transaksi_tinggi)
transaksi_tinggi

,Bulan,Perkembangan_Transaksi
0,1,27.0
1,2,19.0
2,3,24.5
3,4,21.0
4,5,33.5
5,6,28.0
6,7,29.0
7,8,26.5
8,9,43.0
9,10,47.0


In [191]:
import altair as alt

alt.Chart(transaksi_tinggi).mark_line().encode(
    x="Bulan:Q", y="Perkembangan_Transaksi:Q",
)

alt.Chart(...)

In [192]:
from sklearn.linear_model import LinearRegression

X = transaksi_tinggi["Bulan"].values.reshape(-1, 1)
Y = transaksi_tinggi["Perkembangan_Transaksi"].values.reshape(-1, 1)

reg = LinearRegression()
reg.fit(X, Y)

ypred = reg.predict([[13]])
ypred

array([[40.86363636]])